# Introduction à la programmation linéaire en nombres entiers: résolution de Sudoku

Les problèmes linéaires en nombres entiers (PLNE) sont similaires à ceux abordés précédemment avec la contrainte supplémentaire que les variables doivent être des entiers naturelles.

Par exemple une forme standard d'un PLNE est:

\begin{align}
\min & \, \langle c, x \rangle \\
\text{s.c.} \quad 
& Ax = b, \\
& x \ge 0, \\
& x \in \mathbb{Z}^n.
\end{align}

Cette dernière contrainte rend les PLNE non-convexes, ce qui complique leur résolution et demande des technqiues particulières qui ne sont pas abordées cette année. 

Dans ce TP, nous allons utiliser la librairy `pulp` qui permet de résoudre de tels problèmes.


In [ ]:
%pip install pulp 

In [ ]:
import matplotlib.pyplot as plt
import pulp 

## 1. Utilisation de `pulp` sur le problème de transport

Nous considérons à nouveau l'exemple de la distribution de bières du TP2. 

Pour rappel, la modélisation du problème donnait:

### 1. Modélisation du problème

Les variables de decision représentent les volumes de bières (en L) livrés des usines aux bars (représentés par les flèches sur le schéma). Soient $U = \{a,b\}$ et $B = \{1,2,3,4,5\}$ et les variables :
$$
    x_{w,b} \geq 0, \quad \forall w \in W, b \in B
$$

Les contraintes viennent des besoins de chaque bar et des stocks de chaque usine. Le volume de bière distribué depuis une usine ne peut pas excéder son stock. On formule ainsi deux contraintes :
$$
    \sum_{b \in B} x_{w,b} \leq s_w, \quad \forall w \in W
$$
avec $s_a = 1000$ et $s_b = 4000$ représentant le stock de chaque usine.

Pour éviter les pertes de vente, le volume livré à un bar doit être superieur à sa demande qui peut stocker l'excèdent pour la semaine suivante. Ceci permet de formuler 5 contraintes :
$$
    \sum_{w \in W} x_{w,b} \geq d_b, \quad \forall b \in B
$$
avec $d_1 = 500$, $d_2 = 900$, $d_3 = 1800$, $d_4 = 200$, $d_5 = 700$ qui représentent les demandes de chaque bar.

Pour ce problème, nous supposerons que le coût de transport est proportionnel au volume transporté d'une usine à l'autre. Les coûts de transport entre les usine et les bars sont :
$$
\begin{array}{c | c c c c c}
    \hline
    & 1 & 2 & 3 & 4 & 5 \\ \hline
    a & 2 & 4 & 5 & 2 & 1 \\
    b & 3 & 1 & 3 & 2 & 3 \\
\end{array} = (c_{w,b})_{w \in W, b \in B}
$$
en euros par L de bière.
La fonction coût est donc 
$$
    \sum_{w \in W, b \in B} c_{w,b} x_{w,b}
$$




### 2. Modèle `pulp`

La librairy `pulp` permet d'ajouter les contraintes de manière plus simple que la création du dictionnaire en TP2.

On définit les entrepots et leurs productions ainsi que les bars et les demandes.

In [ ]:
# Creates a list of all the supply nodes
Warehouses = ["A", "B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000, "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700,
}

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w, b) for w in Warehouses for b in Bars]

Les couts de transport sont renseignées dans une liste puis dans un dictionnaire de sorte que:
- costs[0][0] = costs_dict["A"]["1"] correspond au cout de transport du warehouse A au bar 1
- costs[1][2] = costs_dict["B"]["3"] correspond au cout de transport du warehouse B au bar 3

In [ ]:
# Creates a list of costs of each transportation path
costs = [  # Bars
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
]

# The cost data is made into a dictionary
costs_dict = pulp.makeDict([Warehouses, Bars], costs, 0)

Le problème est créé dans `pulp` à l'aide de `LpProblem`, les variables à l'aide de `LpVariable` de la façon suivante.


In [ ]:
prob = pulp.LpProblem("Beer", pulp.LpMinimize)

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = pulp.LpVariable.dicts("Route", (Warehouses, Bars), 0, None)

La fonction cout doit être ajoutée en premier dans `prob`. Puis les contraintes. La fonction `pulp.lpSum` permet d'opérer des sommes sur les `pulp.lpVariables`, qui sera traduit en interne sous forme de contrainte.

In [ ]:
# The objective function is added to 'prob' first
prob += (
    pulp.lpSum([vars[w][b] * costs_dict[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += (
        pulp.lpSum([vars[w][b] for b in Bars]) <= supply[w],
        f"Sum_of_Products_out_of_Warehouse_{w}",
    )

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += (
        pulp.lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
        f"Sum_of_Products_into_Bar{b}",
    )



Toutes les contraintes sont ajoutées. Il est maintenant possible de résoudre le problème.

In [ ]:
# The problem data is written to an .lp file
prob.writeLP("BeerDistributionProblem.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total Cost of Transportation = ", pulp.value(prob.objective))

**TODO**: Vérifier que les résultats sont similaires à ceux obtenus au TP précédent. 

Maintenant, supposons que les quantités exprimées ne soient plus des réels (litres) mais des entiers (nombre de caisses). Cette contrainte peut facilement être intégré dans `pulp` en ajoutant `pulp.LpInteger` à déclaration de variables. C'est à dire en remplacant:
- `vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None)`
- par `vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, pulp.LpInteger)`

Relancer la résolution avec cette contrainte et commenter.

# 2. Résolution de Sudoku à l'aide de la PLNE

Un sudoku est un problème consistant en une grille de nombres 9×9 incomplète qui doit être remplie selon plusieurs règles :
- Dans chacun des 9 blocs individuels de 3×3, chacun des nombres de 1 à 9 doit apparaître.
- Dans chaque colonne de la grille 9×9, chacun des nombres de 1 à 9 doit apparaître.
- Dans chaque ligne de la grille 9×9, chacun des nombres de 1 à 9 doit apparaître.

Nous allons résoudre ce Sudoku, mais avec quelques modifications mineurs, le code permettra de résoudre n'importe quel sudoku.

![image](https://coin-or.github.io/pulp/_images/wikisudokuproblem.jpg)

## 1. Modélisation

Pour formuler le sudoku en PL, on pourrait simplement créer une variable pour les 81 cases prenant une valeur entre 1 et 9. 
Néanmoins, il n'existe pas en PL d'opération "différent de" qui permettrait de modéliser les règles précédentes sur les lignes, colonnes, blocs.
Il existe des stratégies assez complexes permettant cette modélisation, mais il existe une facon plus élégante de procéder.

Pour chacune des 81 cases, on associe 9 variables binaires correspondant à chaque nombre entre 1 et 9. On les notera:
$$
    x_{v,i,j} \in \{0,1\}, \quad \forall v,i,j \in \{1,\ldots, 9\}.
$$
Ainsi $x_{v,i,j}$ décrit si la valeur $v$ est présente dans la case $(i,j)$ si elle vaut $1$ et absente sinon. Notons que pour une case fixée $(i,j)$ un seul $v$ peut donner lieu à une varaible $x_{v,i,j}$ à 1 et toutes les autres doivent être à 0.

Le problème du Sudoku consiste à trouver une solution satisfaisant toutes les contraintes. La "qualité" de la solution n'étant pas importante, aucune fonction coût n'est necessaire.

Dans la suite, on notera pour avoirs des notations plus compactes: 
- $I = \{ 1, \ldots, 9 \}$ 
- $B_{k,l} \subset I^2$ le $(k,l)$-ème bloc d'indice de taille $(3 \times 3)$. Le bloc $(1,1)$ est en haut à gauche de la grille, le $(1,2)$ en haut au centre, le $(1,3)$ en haut à droite, le $(2,1)$ au milieu à gauche, etc... Formellemt, cela correspond à $$B_{k,l} = \{ ( 3(k-1) + i, 3(l-1) + j ), \quad i,j \in \{1,2,3\}\}.$$

**TODO:** Formuler les contraintes sur les variables binaires $x_{v,i,j}$ pour qu'elles satifassent les règles du Sudoku. Ecrire une phrase et une équation pour chaque contrainte.


Le code suivant crée les ensemble d'indices $I$, les blocs et les nombres existants sous forme de liste contenant (valeur, indice ligne, indice colonne).

In [ ]:
I = range(1, 10)

Blocs = [
    [(3 * i + k + 1, 3 * j + l + 1) for k in range(3) for l in range(3)]
    for i in range(3)
    for j in range(3)
]

# # The starting numbers are entered as constraints
input_data = [
    (5, 1, 1),
    (6, 2, 1),
    (8, 4, 1),
    (4, 5, 1),
    (7, 6, 1),
    (3, 1, 2),
    (9, 3, 2),
    (6, 7, 2),
    (8, 3, 3),
    (1, 2, 4),
    (8, 5, 4),
    (4, 8, 4),
    (7, 1, 5),
    (9, 2, 5),
    (6, 4, 5),
    (2, 6, 5),
    (1, 8, 5),
    (8, 9, 5),
    (5, 2, 6),
    (3, 5, 6),
    (9, 8, 6),
    (2, 7, 7),
    (6, 3, 8),
    (8, 7, 8),
    (7, 9, 8),
    (3, 4, 9),
    (1, 5, 9),
    (6, 6, 9),
    (5, 8, 9),
]


Le problème `pulp` peut être créé de la facon suivante.

In [ ]:
# The prob variable is created to contain the problem data
prob = pulp.LpProblem("Sudoku_Problem")
# The binary decision variables are created
x_vij = pulp.LpVariable.dicts("Choice", (I, I, I), cat="Binary")


**TODO:** Codes les contraintes

In [ ]:
## contraintes


In [ ]:
# The problem data is written to an .lp file
prob.writeLP("Sudoku.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[prob.status])

La fonction suivante permet d'afficher le résulat. Les chiffres en rouge sont ceux trouvés par l'algorithme.

In [ ]:
from IPython.display import display, Latex

latex = []
latex.append(r"\[")
latex.append(r"\begin{array}{|ccc|ccc|ccc|}")
latex.append(r"\hline")

for r in I:
    row_vals = []
    for c in I:
        for v in I:
            if pulp.value(x_vij[v][r][c]) == 1:
                if not (v,r,c) in input_data:
                    row_vals.append(r"\textcolor{red}{" + str(v) + "}")
                else:
                    row_vals.append(str(v))
                break

    # Join columns and add row ending
    latex.append(" & ".join(row_vals) + r" \\")

    # Horizontal lines after rows 3 and 6
    if r in [3, 6]:
        latex.append(r"\hline")

latex.append(r"\hline")
latex.append(r"\end{array}")
latex.append(r"\]")

display(Latex("\n".join(latex)))

**TODO:** Pour changer de grille de Sudoku, il suffit de changer la liste `input_data`.

In [ ]:
# qualified as one of the hardest Sudoku problem
input_data = [
    (1,1,1),
    (7,1,7),
    (9,1,8),
    (3,2,2),
    (2,2,5),
    (8,2,9),
    (9,3,3),
    (6,3,4),
    (5,4,7),
    (5,4,3),
    (3,4,4),
    (9,4,7),
    (1,5,2),
    (8,5,5),
    (2,5,9),
    (6,6,1),
    (4,6,6),
    (3,7,1),
    (1,7,8),
    (4,8,2),
    (7,8,9),
    (7,9,3),
    (3,9,7)
]